In [17]:
import sys

sys.path.insert(0, '..')

## Setup the model area

In [18]:
from morpheus.modflow.types.discretization.spatial.GridCells import GridCells
from morpheus.modflow.types.discretization import SpatialDiscretization
from morpheus.modflow.types.discretization.spatial import Rotation, LengthUnit, Grid, Crs
from morpheus.modflow.types.geometry import Polygon, LineString

polygon = Polygon(
  type='Polygon',
  coordinates=[[
    (13.922514437551428, 50.964720483303836),
    (13.925250781947113, 50.965228748412386),
    (13.925036413951403, 50.96623732041704),
    (13.92222441026388, 50.96629040370362),
    (13.922514437551428, 50.964720483303836)
  ]]
)

rotation = Rotation(0)
length_unit = LengthUnit.meters()
x_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]
y_coordinates = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

grid = Grid.from_polygon_with_relative_coordinates(
  polygon=polygon,
  x_coordinates=x_coordinates,
  y_coordinates=y_coordinates,
  rotation=rotation,
)

spatial_discretization = SpatialDiscretization(
  geometry=polygon,
  grid=grid,
  affected_cells=GridCells.from_polygon(polygon=polygon, grid=grid),
  crs=Crs.from_str('EPSG:4326')
)

spatial_discretization



SpatialDiscretization(geometry=Polygon(coordinates=[[(13.922514437551428, 50.964720483303836), (13.925250781947113, 50.965228748412386), (13.925036413951403, 50.96623732041704), (13.92222441026388, 50.96629040370362), (13.922514437551428, 50.964720483303836)]], type='Polygon'), grid=Grid(origin=Point(coordinates=(13.922224410263878, 50.9662904037036), type='Point'), x_distances=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], y_distances=[27.749448566045615, 27.749448566045615, 27.7494485660456, 27.74944856604563, 27.7494485660456, 27.7494485660456, 27.7494485660456, 27.749448566045658, 27.7494485660456, 27.7494485660456], rotation=Rotation(value=0), length_unit=LengthUnit(unit=2)), affected_cells=GridCells(shape=(10, 10), data=[GridCell(x=0, y=0, value=True), GridCell(x=0, y=1, value=True), GridCell(x=0, y=2, value=True), GridCell(x=0, y=3, value=

## Setup Time Discretization

In [19]:
from morpheus.modflow.types.discretization.time.TimeUnit import TimeUnit
from morpheus.modflow.types.discretization.time.Stressperiods import StartDateTime, StressPeriodCollection, \
  StressPeriod, NumberOfTimeSteps, TimeStepMultiplier, IsSteadyState
from morpheus.modflow.types.discretization.time import TimeDiscretization
from datetime import datetime

time_discretization = TimeDiscretization(
  start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
  end_date_time=StartDateTime.from_datetime(datetime(2020, 12, 31)),
  stress_periods=StressPeriodCollection([
    StressPeriod(
      start_date_time=StartDateTime.from_datetime(datetime(2020, 1, 1)),
      number_of_time_steps=NumberOfTimeSteps(1),
      time_step_multiplier=TimeStepMultiplier(1),
      steady_state=IsSteadyState.yes()
    ),
  ]),
  time_unit=TimeUnit.days()
)

time_discretization

TimeDiscretization(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), end_date_time=StartDateTime(value=datetime.datetime(2020, 12, 31, 0, 0)), stress_periods=StressPeriodCollection(values=[StressPeriod(start_date_time=StartDateTime(value=datetime.datetime(2020, 1, 1, 0, 0)), number_of_time_steps=NumberOfTimeSteps(value=1), time_step_multiplier=TimeStepMultiplier(value=1), steady_state=IsSteadyState(value=True))]), time_unit=TimeUnit(unit=4))

# Setup Modflow Model

In [20]:
from morpheus.modflow.types.ModflowModel import ModflowModel

modflow_model = ModflowModel.new()
modflow_model = modflow_model.with_updated_time_discretization(time_discretization)
modflow_model = modflow_model.with_updated_spatial_discretization(spatial_discretization=spatial_discretization)

In [21]:
import folium

m = folium.Map(tiles="cartodbpositron", crs="EPSG3857", zoom_start=12, location=[50.965, 13.922])
m.add_child(folium.GeoJson(polygon.as_geojson()))

m.fit_bounds(m.get_bounds())
m

# Setup constant head boundary with 1 observation

In [22]:
from morpheus.modflow.types.boundaries.Observation import ObservationId
from morpheus.modflow.types.geometry import Point
from morpheus.modflow.types.boundaries.Boundary import BoundaryName, ConstantHeadBoundary
from morpheus.modflow.types.soil_model import LayerId
from morpheus.modflow.types.boundaries.ConstantHeadObservation import ConstantHeadObservation, ConstantHeadRawDataItem, \
  HeadValue

layers_list = [LayerId.new()]
constant_head = ConstantHeadBoundary.from_geometry(
  name=BoundaryName('constant_head'),
  geometry=LineString([
    (13.9223, 50.9662),
    (13.9250, 50.96615),
  ]),
  grid=grid,
  affected_layers=[modflow_model.soil_model.layers[0].id],
  observations=[
    ConstantHeadObservation(
      observation_id=ObservationId.new(),
      geometry=Point((13.9223, 50.9662)),
      raw_data=[
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2019, 1, 1)),
          head=HeadValue.from_float(100),
        ),
        ConstantHeadRawDataItem(
          date_time=StartDateTime.from_datetime(datetime(2021, 12, 31)),
          head=HeadValue.from_float(100),
        ),
      ]
    )
  ]
)

m.add_child(folium.GeoJson(constant_head.as_geojson()))
m.add_child(folium.GeoJson(constant_head.affected_cells.as_geojson(grid).as_geojson()))
for constant_head.observation in constant_head.observations:
  m.add_child(folium.GeoJson(constant_head.observation.as_geojson()))
m.fit_bounds(m.get_bounds())
m

In [23]:
from morpheus.modflow.types.boundaries.Boundary import BoundaryCollection

boundaries = BoundaryCollection.new()
boundaries.add_boundary(constant_head)
modflow_model = modflow_model.with_updated_boundaries(boundaries=boundaries)

## Calculate the model asynchronously with celery

In [24]:
from morpheus.modflow.infrastructure.calculation.modflow_2005.Mf2005CalculationEngineSettings import \
  Mf2005CalculationEngineSettings

calculation_engine_setting = Mf2005CalculationEngineSettings.default()
calculation_engine_setting

Mf2005CalculationEngineSettings(packages=PackageData(oc=OcPackageData(ihedfm=0, iddnfm=0, chedfm=None, cddnfm=None, cboufm=None, compact=True, stress_period_data=None, extension=['oc', 'hds', 'ddn', 'cbc', 'ibo'], unitnumber=None, filenames=None, label='OC'), flow_package=FlowPackage(type='lpf'), solver_package=SolverPackageData(type='pcg', data=PcgPackageData(mxiter=50, iter1=30, npcond=1, hclose=1e-05, rclose=1e-05, relax=1.0, nbpol=0, iprpcg=0, mutpcg=3, damp=1.0, dampt=1.0, ihcofadd=0, extension='pcg', unitnumber=None, filenames=None))))

In [25]:
from morpheus.modflow.infrastructure.persistence.CalculationRepository import calculation_repository
from morpheus.modflow.types.calculation.CalculationProfile import CalculationProfile, CalculationEngineType
from morpheus.modflow.types.calculation.Calculation import Calculation

calculation = Calculation.new(
  modflow_model=modflow_model, 
  calculation_profile=CalculationProfile.new(CalculationEngineType.MF2005, calculation_engine_setting.to_dict())
)
calculation_repository.save_calculation(calculation)
calculation.calculation_id

CalculationId(value='7299ee12-1b1c-49bd-9c9b-a90f25ac4d23')

In [26]:
from morpheus.modflow.tasks import run_calculation_by_id

result = run_calculation_by_id.delay(calculation.calculation_id.to_str())
result_state = result.state
result_id = result.id
{'id': result_id, 'state': result_state}

{'id': '8ee3c788-93c1-4fd3-b844-f72ae3323f91', 'state': 'PENDING'}

In [27]:
from celery.result import AsyncResult

result = AsyncResult(id=result_id)
{
  'id': result.id,
  'state': result.state,
  'ready': result.ready(),
  'successful': result.successful(),
  'value': result.result if result.ready() else None,
}

{'id': '8ee3c788-93c1-4fd3-b844-f72ae3323f91',
 'state': 'PENDING',
 'ready': False,
 'successful': False,
 'value': None}

In [28]:
calculation = calculation_repository.get_calculation(calculation.calculation_id)
calculation

Calculation(calculation_id=CalculationId(value='7299ee12-1b1c-49bd-9c9b-a90f25ac4d23'), modflow_model=ModflowModel(model_id=ModelId(value='6130c292-adac-4213-b36b-3f43b72cd28f'), spatial_discretization=SpatialDiscretization(geometry=Polygon(coordinates=[[[13.922514437551428, 50.964720483303836], [13.925250781947113, 50.965228748412386], [13.925036413951403, 50.96623732041704], [13.92222441026388, 50.96629040370362], [13.922514437551428, 50.964720483303836]]], type='Polygon'), grid=Grid(origin=Point(coordinates=[13.922224410263878, 50.9662904037036], type='Point'), x_distances=[33.68941547288559, 33.68941547288559, 33.6894154728856, 33.689415472885585, 33.6894154728856, 33.6894154728856, 33.68941547288557, 33.6894154728856, 33.68941547288563, 33.68941547288557], y_distances=[27.749448566045615, 27.749448566045615, 27.7494485660456, 27.74944856604563, 27.7494485660456, 27.7494485660456, 27.7494485660456, 27.749448566045658, 27.7494485660456, 27.7494485660456], rotation=Rotation(value=0),

In [29]:
calculation.calculation_state

<CalculationState.FINISHED: 'finished'>

In [30]:
calculation.calculation_log

CalculationLog(log=['', '                                  MODFLOW-2005     ', '    U.S. GEOLOGICAL SURVEY MODULAR FINITE-DIFFERENCE GROUND-WATER FLOW MODEL', '                             Version 1.12.00 2/3/2017                        ', '', ' Using NAME file: 6130c292-adac-4213-b36b-3f43b72cd28f.nam ', ' Run start date and time (yyyy/mm/dd hh:mm:ss): 2023/12/13 19:14:44', '', ' Solving:  Stress period:     1    Time step:     1    Ground-Water Flow Eqn.', ' Run end date and time (yyyy/mm/dd hh:mm:ss): 2023/12/13 19:14:44', ' Elapsed run time:  0.002 Seconds', '', '  Normal termination of simulation'])

In [31]:
calculation.calculation_result

CalculationResult(type=<CalculationResultType.SUCCESS: 'success'>, message='', files=['6130c292-adac-4213-b36b-3f43b72cd28f.hds', '6130c292-adac-4213-b36b-3f43b72cd28f.pcg', '6130c292-adac-4213-b36b-3f43b72cd28f.oc', '6130c292-adac-4213-b36b-3f43b72cd28f.dis', '6130c292-adac-4213-b36b-3f43b72cd28f.bas', '6130c292-adac-4213-b36b-3f43b72cd28f.list', '6130c292-adac-4213-b36b-3f43b72cd28f.nam', '6130c292-adac-4213-b36b-3f43b72cd28f.chd', '6130c292-adac-4213-b36b-3f43b72cd28f.lpf'], head_results=None, drawdown_results=None, budget_results=AvailableResults(times=[365.0], kstpkper=[[0, 0]], number_of_layers=0), concentration_results=None)